In [ ]:
### General imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
from emukit.test_functions import forrester_function
from emukit.core.loop.user_function import UserFunctionWrapper
from emukit.core import ContinuousParameter, ParameterSpace

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

# import emukit
import GPy
# from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
# from emukit.core.initial_designs.random_design import RandomDesign
# from emukit.core.initial_designs.latin_design import LatinDesign
from GPy.models import GPRegression
from emukit.model_wrappers import GPyModelWrapper

# from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement, \
                                                      NegativeLowerConfidenceBound, \
                                                      MaxValueEntropySearch, \
                                                      ProbabilityOfImprovement
# from emukit.core.acquisition import IntegratedHyperParameterAcquisition

# from rgpe import compute_rank_weights

In [ ]:
def find_y_by_x(x_input, X_all, Y_all):  #一通过x找到对应的y的函数
    # 确保 x_input 是一个二维数组
    if len(x_input.shape) == 1:
        x_input = x_input.reshape(1, -1)

    # 初始化一个列表来存储所有结果
    all_results = []

    # 遍历 x_input 中的每个特征向量
    for x in x_input:
        # 使用 NumPy 的函数 argwhere 来查找 x 在 X 中的位置
        indices = np.argwhere(np.all(X_all == x, axis=1))

        # 初始化一个列表来存储匹配的 Y 值
        matching_y_values = []

        # 遍历匹配的位置
        for index in indices:
            match_index = index[0]
            matching_y_values.append(Y_all[match_index])

        all_results.append(matching_y_values)
    
    all_results = np.array(all_results)
    return all_results



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs.random_design import RandomDesign
from emukit.core.initial_designs.latin_design import LatinDesign

In [ ]:
def x_normalizer(X, var_array):
    
    def max_min_scaler(x, x_max, x_min):
        return (x-x_min)/(x_max-x_min)
    x_norm = []
    for x in (X):
           x_norm.append([max_min_scaler(x[i], 
                                         max(var_array[i]), 
                                         min(var_array[i])) for i in range(len(x))])
            
    return x_norm

def x_denormalizer(x_norm, var_array):
    
    def max_min_rescaler(x, x_max, x_min):
        return x*(x_max-x_min)+x_min
    x_original = []
    for x in (x_norm):
           x_original.append([max_min_rescaler(x[i], 
                                         max(var_array[i]), 
                                         min(var_array[i])) for i in range(len(x))])
            
    return x_original


import numpy as np
from scipy.spatial.distance import cdist


# def get_closest_array(x_init, X_all):
#     # 计算 x_init 中每个点与 X_all 中所有点之间的距离
#     distances = cdist(x_init, X_all)   
#     # 找到每个点最近的索引
#     closest_indices = np.argmin(distances, axis=1)    
#     # 根据索引获取最近的点
#     closest_array = X_all[closest_indices]  
#     return closest_array

def get_closest_array(x_init, X_all):
    X_all_copy = X_all.copy()
    closest_array = []
    
    for x in x_init:
        # 计算 x 与 X_all_copy 中所有点之间的距离
        distances = cdist([x], X_all_copy)[0]
        # 找到最近的索引
        closest_index = np.argmin(distances)
        # 根据索引获取最近的点
        closest_point = X_all_copy[closest_index]
        # 将最近的点添加到结果数组中
        closest_array.append(closest_point)
        # 从 X_all_copy 中移除这个点
        X_all_copy = np.delete(X_all_copy, closest_index, axis=0)
        
    return np.array(closest_array)


# # 计算两个点之间的欧几里得距离
# def euclidean_distance(point1, point2):
#     return np.sqrt(np.sum((point1 - point2) ** 2))

# # 找到X_all中与x_init每个元素最近的点
# def get_closest_array(x_init, X_all):
#     closest_array = []
#     for point in x_init:
#         distances = np.array([euclidean_distance(point, x) for x in X_all])
#         closest_idx = distances.argmin()
#         closest_array.append(X_all[closest_idx])
#     return np.array(closest_array)

In [ ]:
def calculate_distances(X):
    # 计算每个向量与自己的欧氏距离
    self_distances = np.linalg.norm(X[:, np.newaxis, :] - X, axis=-1)

    # 将对角线上的距离设置为无穷大，排除自身与自身的距离
    np.fill_diagonal(self_distances, np.inf)

    # 计算除了自己之外的其他向量与每个向量的欧氏距离
    other_distances = np.min(self_distances, axis=1)

    # 将结果转换为二维数组
    other_distances = other_distances.reshape(-1, 1)

    # 计算均值

    mean_distance = np.mean(other_distances)

    return mean_distance

def create_latin_design(X_all):
 
    x_array = [X_all[:, i] for i in range(X_all.shape[1])]
    
    num_columns = X_all.shape[1]
    parameter_space = ParameterSpace([ContinuousParameter(f'x{i+1}', 0, 1) for i in range(num_columns)])
    design = LatinDesign(parameter_space)
    
    return x_array , design

# 导入数据

In [ ]:
import pandas as pd
import os

# folder_path = r'C:\Users\13282\Desktop\papercode\data\Goldstein' 
# file_path = os.path.join(folder_path, 'goldstein_data.xlsx')
# df = pd.read_excel(file_path)

# x_all = df.iloc[:, :-1].values  
# y_all = df.iloc[:, -1].values   



# folder_path = r'C:\Users\13282\Desktop\papercode\data\Aryl\original_processed' 
# file_path = os.path.join(folder_path, 'aryl-15.xlsx')  # y 记得加负号 
# 'aryl-1.xlsx'   'all_data.xlsx'  'aryl-1.xlsx' 

# folder_path = r'C:\Users\13282\Desktop\papercode\data\Alloy\matminner_processed' 
# file_path = os.path.join(folder_path, 's1_data.xlsx')  # y 记得加负号 ，如果需要原始值，需要反log回去
# Ti  Virtual_ti  s1_data  s1+s2_data
# folder_path = r'C:\Users\13282\Desktop\papercode\data\Alloy\no_comp' 
# file_path = os.path.join(folder_path, 'Ni.xlsx')  # y 记得加负号 ，如果需要原始值，需要反log回去

df = pd.read_excel(file_path)
x_all = df.iloc[:, :-1].values  
y_all = -df.iloc[:, -1].values   
nan_indices = np.isnan(y_all)
true_indices = np.where(nan_indices)[0]
print("nan 的索引:", true_indices)

x_all = x_all[~nan_indices.flatten()]
y_all = y_all[~nan_indices.flatten()]

print(np.min(y_all))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=43)

X_all, Y_all = [X_train, Y_train.reshape(-1)]

Y_test = Y_test.reshape(-1, 1)

print(X_all.shape,Y_all.shape)
print(X_test.shape,Y_test.shape)